## Imports

In [39]:
import os
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold, cross_val_score


# Data

In [40]:
data = pd.read_csv('new_Exam_Score_Prediction.csv')

y = data['exam_score']
X = data.drop('exam_score', axis=1)

data

,age,gender,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,facility_rating,exam_difficulty,exam_score,diploma,group_study,self_study,online_videos,coaching,mixed
0,17,1,2.78,92.9,1,7.4,0,0,2,58.9,1,0,0,0,1,0
1,22,1,7.88,76.8,1,8.5,0,2,1,90.3,0,0,0,0,1,0
2,20,0,0.89,71.6,1,9.8,0,0,1,43.7,1,0,0,0,1,0
3,23,1,3.48,65.4,1,4.2,2,0,1,58.2,0,0,0,0,0,1
4,17,0,1.35,69.0,1,7.4,1,2,2,53.7,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13269,17,1,5.74,81.4,1,7.5,2,2,2,95.0,0,0,0,1,0,0
13270,19,0,3.28,73.3,1,9.5,2,2,0,93.9,0,1,0,0,0,0
13271,18,1,3.71,41.6,0,5.9,1,1,1,60.9,0,0,0,0,1,0
13272,19,1,4.60,76.3,0,6.1,2,1,1,79.0,0,0,1,0,0,0


# Models

In [41]:
PATH = 'pipes'
pipe_names = os.listdir(PATH)[:-1] #Gitignore file to not pushing the models

pipes = {}

for name in pipe_names:
    with open(os.path.join(PATH, name), 'rb') as f:
        pipes[name.strip('.pkl')] = pickle.load(f)

pipes

{'XSB': Pipeline(steps=[('scaler', StandardScaler()),
                 ('features_selector',
                  SelectKBest(k=14,
                              score_func=<function f_regression at 0x7f26ce1c0680>)),
                 ('regressor',
                  XGBRegressor(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.5103824786422447, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, e...
                               gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.03796623227630468, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=

# Friedman test

In [44]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

#TODO: Get models score


#TODO Friedman TEST



